Link to Drive for CDs dataset: https://drive.google.com/drive/folders/1t2Y24NSpGlT2M29zJUDvvEg2KfP0JqkN?usp=share_link

# Preprocessing...

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import json
import matplotlib.pyplot as plt
import os
import tqdm
import pickle
from pathlib import Path
from torch.utils.data import DataLoader
from sklearn.metrics import ndcg_score
np.random.seed(0)

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/CDs_and_Vinyl.json.gz

--2022-06-15 04:26:28--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/CDs_and_Vinyl.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1172666826 (1.1G) [application/octet-stream]
Saving to: ‘CDs_and_Vinyl.json.gz’

CDs_and_Vinyl.json. 100%[===================>]   1.09G  87.5MB/s    in 13s     

2022-06-15 04:26:41 (85.5 MB/s) - ‘CDs_and_Vinyl.json.gz’ saved [1172666826/1172666826]



In [ ]:
!gunzip CDs_and_Vinyl.json.gz

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_CDs_and_Vinyl.json.gz

--2022-06-15 04:26:41--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_CDs_and_Vinyl.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 161716387 (154M) [application/octet-stream]
Saving to: ‘meta_CDs_and_Vinyl.json.gz’

meta_CDs_and_Vinyl. 100%[===================>] 154.22M  79.5MB/s    in 1.9s    

2022-06-15 04:26:43 (79.5 MB/s) - ‘meta_CDs_and_Vinyl.json.gz’ saved [161716387/161716387]



In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import FreqDist
import re
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

In [ ]:
save_path='/content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/cds/'

In [ ]:
df_review=pd.read_csv(save_path+'df_reviews.csv')

In [ ]:
df_meta=pd.read_csv(save_path+'df_meta.csv')

In [ ]:
df_review.drop(df_review.index[~df_review['asin'].isin(df_meta['asin'])], inplace=True) 

In [ ]:
df_bert_sentiment=pd.read_csv(save_path+'dframe_bert_sentiments.csv')

In [ ]:
df_bert_sentiment.drop(df_bert_sentiment.index[~df_bert_sentiment['item_id'].isin(df_meta['asin'])], inplace=True)

In [ ]:
df_bert_sentiment

,Unnamed: 0,user_id,item_id,feature,sentiment,vect
0,0,AHO9V8YUQ2G3Q,B0004OPNTA,item good,1,"[-0.061487213, -0.21710062, 0.457771, 0.309799..."
1,1,A2MXCDMK91S143,B0004OPNTA,item perfect,1,"[0.43000737, 0.010507888, -0.03303118, 0.37484..."
2,2,A2MXCDMK91S143,B0004OPNTA,item perfect,1,"[0.49877173, 0.21823983, 0.30037588, 0.4995622..."
3,3,A2FD834RITVVXH,B0004OPNTA,price great,1,"[-0.14932808, -0.14713731, 0.31191912, 0.29198..."
4,4,A39EN5XP8LDVX8,B0006TIA8Y,brand new,1,"[0.16046865, -0.15162839, 0.1722278, -0.227062..."
...,...,...,...,...,...,...
36772,36772,AMM66EYPMRWL6,B01HCH03HS,protection good,1,"[-0.2639229, 0.15404528, 0.9183527, -0.1205100..."
36773,36773,AUZ1JF6JPUS99,B01HCH03HS,phone protected,1,"[-0.4061425, -0.007457112, -0.060606517, -0.30..."
36774,36774,A3OS5Q3V8BCSVP,B01HCH03HS,product great,1,"[0.49021837, -0.43521026, 0.2531106, 0.4747344..."
36775,36775,AHA390NSTV3VD,B01HCH03HS,case good,1,"[-0.17711374, -0.5704334, 0.14784569, 0.202380..."


In [ ]:
sentires_dir='/content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/cds/CDs_and_Vinyl'
test_length=5
sample_ratio=2
# val_length=1
neg_length=50
dataset='cds_and_vinyl'
save_path='/content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/cds/'

In [ ]:
def get_user_item_dict(df_bert_sentiment,items_list):
  user_dict = {}
  item_dict = {}
  # removed_users=['A1ULCCHD1QNOS5','A1ZS098EKPVT8F','A1K1WK6I122RX2','A2PAFKGAUSBMIE','AAOYA0DKWED4W','AJS9Q2JYS3DLJ','ADOF1VKGDCBWF','A3G11XDKGXZT9Q','A1R233YLWSRBTC','A3VH9QMH2UTX9D','A15DZOS6KVANQH']    
  for index, row in df_bert_sentiment.iterrows():
    user=row['user_id']
    item=row['item_id']
    if item in items_list:
      if user not in user_dict:
          user_dict[user] = [item]
      else:
          user_dict[user].append(item)
      if item not in item_dict:
          item_dict[item] = [user]
      else:
          item_dict[item].append(user)
  return user_dict,item_dict

In [ ]:
df_meta.loc[df_meta['feature'].str.len()<15]

,Unnamed: 0,description,title,feature,rank,price,asin
342,4082,"['Blueant product, not a fake']",SUPERTOOTH LIGHT BLUETOOTH MOBILE PHONE HANDSF...,['brand new'],"['>#3,652,634 in Cell Phones & Accessories (Se...",$149.99,B000FQ4GT0
376,4470,['Do it Swivel Glide - For installation in hol...,Swivel Glide,['China'],"['>#1,892,113 in Tools & Home Improvement (See...",$6.99,B000HE4TPG
483,5453,['Lucky Line Leather Belt Hook Key Ring - Made...,Lucky Line 45301 Leather Belt Hook,['China'],"['>#1,885,591 in Cell Phones & Accessories (Se...",$6.71,B000LNU0NI
889,9733,['Mot W385 Standard 940 mAh lithium ion Battery'],Motorola K1m W220 W385 Z6m Z6tv BT51 Battery,['Batteries'],"['>#101,575 in Cell Phones & Accessories (See ...",$4.47,B001680REO
958,10463,['LG VX8550 Chocolate Standard Battery Black'],LG VX8550 Chocolate Std Battery Black,['Batteries'],"['>#415,269 in Cell Phones & Accessories (See ...",$5.48,B0019F6WXE
...,...,...,...,...,...,...,...
104134,585106,"['perfectly fit, easy to install, covering the...","Galaxy S7 Active Case, DuroCase Hybrid Dual La...",['Discovery'],"['>#1,526,176 in Cell Phones & Accessories (Se...",$11.29,B01H0YUGU2
104157,585147,"['perfectly fit, easy to install, covering the...","LG Stylo 2 Plus Case / LG Stylus 2 Plus Case, ...",['Discovery'],"['>#347,482 in Cell Phones & Accessories (See ...",$11.29,B01H12AL12
104173,585200,"['perfectly fit, easy to install, covering the...","LG Stylo 2 Plus Case / LG Stylus 2 Plus Case, ...",['Discovery'],"['>#2,052,064 in Cell Phones & Accessories (Se...",$11.29,B01H149Z80
104929,588361,"['perfectly fit, easy to install, covering the...",DKmagic 6PCS Mini Earphone SD Card Macarons Ba...,['Discovery'],"['>#1,220,879 in Cell Phones & Accessories (Se...",$2.96,B01HCSDRHA


In [ ]:
def get_average_vect_train(df,not_in_columns):
  lists=df.loc[:, ~df.columns.isin([not_in_columns])].values
  words=df[not_in_columns].values
  # print(words)
  vects=[sub_list[0] for sub_list in lists]
  average=np.average(np.array(vects),axis=0)
  # print(average)
  return average,list(words)

In [ ]:
import os

In [ ]:
from typing_extensions import final
def get_item_matrix(inv_item_name_dict,item_name_dict,user_name_dict,df_bert_sentiment,df_meta,items_list):
  print('----- get items matrix -------')
  # normalized_rank=get_normalized_rank(df_meta)
  item_quality_matrix = np.zeros((len(inv_item_name_dict), 1536))
  print((len(inv_item_name_dict)))
  i=0
  features={}
  for  item in inv_item_name_dict.keys():
    # user_id= inv_user_name_dict[user]
    features[item]=[]
    if i%200==0:
      print(i)
    i+=1
    # if i <4455:
    #   continue
    item_id= inv_item_name_dict[item]
    final_vector=[]
    if os.path.exists(save_path+'descriptions_bert/'+'df_bert_desc_{}.json'.format(item_id)) :
      df_vect_desc= pd.read_json(save_path+'descriptions_bert/'+'df_bert_desc_{}.json'.format(item_id))
      average_vect_desc,words_desc=get_average_vect_train(df_vect_desc,'description_words')

      df_vect_title= pd.read_json(save_path+'titles_bert/'+'df_bert_title_{}.json'.format(item_id))
      average_vect_title,words_title=get_average_vect_train(df_vect_title,'title_words')

      # df_vect_feature= pd.read_json(save_path+'features_bert/'+'df_bert_feature_{}.json'.format(item_id))
      # average_vect_feature,words_feature=get_average_vect_train(df_vect_feature,'feature_words')

      # average_bert_sentiment,words_senti=get_average_bert_sentiment_train(item_id,df_bert_sentiment,user_name_dict,item_name_dict,'item_id',items_list)
      
      # rank=normalized_rank[item_id]
      features[item].append(words_desc)
      features[item].append(words_title)
      # features[item].append(words_feature)
      # features[item].append(words_senti)
      # features[item].append([rank])
      final_vector=list(average_vect_desc)
      final_vector+=list(average_vect_title)
      # final_vector+=list(average_vect_feature)
      # final_vector+=list(average_bert_sentiment)
      # final_vector+=[rank]
      
      if len(final_vector)>1536:
          print(len(list(average_vect_desc)))
          print(len(list(average_vect_title)))
          # print(len(list(average_vect_feature)))
          # print(len(list(average_bert_sentiment)))
          print(item_id)
      item_quality_matrix[item]=final_vector
      # print(item_quality_matrix)

  item_quality_matrix = np.array(item_quality_matrix, dtype='float32')
  return item_quality_matrix,features

In [ ]:
def get_user_matrix(df_meta,item_matrix,items_features,inv_user_name_dict,item_name_dict,user_name_dict,df_bert_sentiment,items_list):
  # normalized_rank=get_normalized_rank(df_meta)
  user_quality_matrix = np.zeros((len(inv_user_name_dict), 1536))
  i=0
  print('----- get users matrix -------')
  print((len(inv_user_name_dict)))
  user_features={}
  for  user in inv_user_name_dict.keys():
    if i%1000==0:
      print(i)
    i+=1
    user_id= inv_user_name_dict[user]
    # average_bert_sentiment,words_senti=get_average_bert_sentiment_train(user_id,df_bert_sentiment,user_name_dict,item_name_dict,'user_id',items_list)
    # item_id= inv_item_name_dict[item]
    final_vector=[]
    items_interacted=df_bert_sentiment[df_bert_sentiment['user_id']==user_id]['item_id'].values
    user_features[user]=[]
    for item_id in items_interacted:
      if item_id in items_list:
        item=item_name_dict[item_id]
        if os.path.exists(save_path+'descriptions_bert/'+'df_bert_desc_{}.json'.format(item_id)):
          final_vector.append(item_matrix[item])
          user_features[user].append(items_features[item])
    
    final_average=np.average(final_vector,axis=0)
    # final_average[2304:3840]=average_bert_sentiment
    
    # if np.isnan(np.sum(final_average)):
    #   print("YEEEEEEEEESSS")
    if ~np.isnan(np.sum(final_average)):
      user_quality_matrix[user]=final_average
    else:
      print(user_id)
  user_quality_matrix = np.array(user_quality_matrix, dtype='float32')
  return user_quality_matrix,user_features

In [ ]:
def sample_training_pairs(user, training_items, item_set, sample_ratio=10):
    positive_items = set(training_items)
    negative_items = set()
    for item in item_set:
        if item not in positive_items:
            negative_items.add(item)
    neg_length = len(positive_items) * sample_ratio
    negative_items = np.random.choice(np.array(list(negative_items)), neg_length, replace=False)
    train_pairs = []
    for p_item in positive_items:
        train_pairs.append([user, p_item, 1])
    for n_item in negative_items:
        train_pairs.append([user, n_item, 0])
    return train_pairs

In [ ]:
import glob
def get_items_list():
  items_list=[]
  names=glob.glob(save_path+"titles_bert/*.json")
  for name in names:
    items_list.append(name.split('/')[-1].split('_')[-1][0:-5])
  return items_list

In [ ]:
final_items = []
final_users=[]
training_pairs = np.loadtxt(save_path+'training_data.txt',dtype=str)
for pair in training_pairs:
  final_items.append(pair[1])
  final_users.append(pair[0])
with open(save_path+'test_data.pickle', 'rb') as f:
  test_pairs= pickle.load(f)
  for user_id in test_pairs.keys():
    final_users.append(user_id)
    items_ids=test_pairs[user_id][0]
    final_items+=items_ids
final_items=list(set(final_items))
final_users=list(set(final_users))

In [ ]:
print(len(final_items))
print(len(final_users))

12087
6934


In [ ]:
from re import S
import torch
import numpy as np
import json
import pickle
# from torch._C import R
import tqdm
from torch.random import seed


class AmazonDataset():
    def __init__(self):
        super().__init__()

        self.user_name_dict = {}  # rename users to integer names
        self.item_name_dict = {}
        self.feature_name_dict = {}

        self.features = {}  # feature list
        self.users = []
        self.items = []

        # the interacted items for each user, sorted with date {user:[i1, i2, i3, ...], user:[i1, i2, i3, ...]}
        self.user_hist_inter_dict = {}
        # the interacted users for each item
        self.item_hist_inter_dict = {}  

        self.user_num = None
        self.item_num = None
        self.feature_num = 1536# number of features

        self.user_feature_matrix = None  # user aspect attention matrix
        self.item_feature_matrix = None  # item aspect quality matrix

        self.training_data = None
        self.test_data = None
        self.pre_processing()
        self.get_user_item_feature_matrix()
        self.sample_training()  # sample training data, for traning BPR loss
        self.sample_test()  # sample test data

    def pre_processing(self,):
        self.items_list=get_items_list()
        user_dict, item_dict = get_user_item_dict(df_bert_sentiment,self.items_list)  # not sorted with time
        print(len(item_dict))
        user_item_date_dict = {}   # {(user, item): date, (user, item): date ...}  # used to remove duplicate
        # removed_users=['A1ULCCHD1QNOS5','A1ZS098EKPVT8F','A1K1WK6I122RX2','A2PAFKGAUSBMIE','AAOYA0DKWED4W','AJS9Q2JYS3DLJ','ADOF1VKGDCBWF','A3G11XDKGXZT9Q','A1R233YLWSRBTC','A3VH9QMH2UTX9D','A15DZOS6KVANQH']
        for i, row in df_review.iterrows():
            user = row['reviewerID']
            # if user not in removed_users:
            item = row['asin']
            date = row['unixReviewTime']
            if item in self.items_list:
              if user in user_dict and item in user_dict[user] and (user, item) not in user_item_date_dict:
                  user_item_date_dict[(user, item)] = date
      
        # rename users, items, and features to integer names
        user_name_dict = {}
        item_name_dict = {}
        # feature_name_dict = {}
        # features = get_feature_list(df_bert_sentiment,df_bert_desc,)
        
        count = 0
        for user in user_dict:
            if user not in user_name_dict:
                user_name_dict[user] = count
                count += 1
        count = 0
        for item in item_dict:
            if item not in item_name_dict:
                item_name_dict[item] = count
                count += 1
        self.inv_user_name_dict = {v: k for k, v in user_name_dict.items()}
        self.inv_item_name_dict = {v: k for k, v in item_name_dict.items()}
        
        # for i in range(len(sentiment_data)):
        #     sentiment_data[i][0] = user_name_dict[sentiment_data[i][0]]
        #     sentiment_data[i][1] = item_name_dict[sentiment_data[i][1]]
        #     for j in range(len(sentiment_data[i]) - 2):
        #         sentiment_data[i][j+2][0] = feature_name_dict[sentiment_data[i][j + 2][0]]

        renamed_user_item_date_dict = {}
        for key, value in user_item_date_dict.items():
            renamed_user_item_date_dict[user_name_dict[key[0]], item_name_dict[key[1]]] = value

        # sort with date
        renamed_user_item_date_dict  = dict(sorted(renamed_user_item_date_dict .items(), key=lambda item: item[1]))

        user_hist_inter_dict = {}  # {"u1": [i1, i2, i3, ...], "u2": [i1, i2, i3, ...]}, sort with time
        item_hist_inter_dict = {}
        # ranked_user_item_dict = {}  # {"u1": [i1, i2, i3, ...], "u2": [i1, i2, i3, ...]}
        for key, value in renamed_user_item_date_dict.items():
            user = key[0]
            item = key[1]
            if user not in user_hist_inter_dict:
                user_hist_inter_dict[user] = [item]
            else:
                user_hist_inter_dict[user].append(item)
            if item not in item_hist_inter_dict:
                item_hist_inter_dict[item] = [user]
            else:
                item_hist_inter_dict[item].append(user)

        user_hist_inter_dict = dict(sorted(user_hist_inter_dict.items()))
        item_hist_inter_dict = dict(sorted(item_hist_inter_dict.items()))

        users = list(user_hist_inter_dict.keys())
        items = list(item_hist_inter_dict.keys())

        self.user_name_dict = user_name_dict
        self.item_name_dict = item_name_dict
        self.user_hist_inter_dict = user_hist_inter_dict
        self.item_hist_inter_dict = item_hist_inter_dict
        self.users = users
        self.items = items
        self.user_num = len(users)
        self.item_num = len(items)
        return True
    
    def get_user_item_feature_matrix(self,):
        # exclude test data from the sentiment data to construct matrix
        train_u_i_set = set()
        for user, items in self.user_hist_inter_dict.items():
            items = items[:-test_length]
            for item in items:
                train_u_i_set.add((user, item))

        self.item_feature_matrix,self.item_features = get_item_matrix(self.inv_item_name_dict,self.item_name_dict,self.user_name_dict,df_bert_sentiment,df_meta,self.items_list)
        self.user_feature_matrix,self.user_features = get_user_matrix(df_meta,self.item_feature_matrix,self.item_features,self.inv_user_name_dict,self.item_name_dict,self.user_name_dict,df_bert_sentiment,self.items_list)
        
        return True
    def sample_training(self):
        print('======================= sample training data =======================')
        # print(self.user_feature_matrix.shape, self.item_feature_matrix.shape)
        training_data = []
        training_pairs = np.loadtxt(save_path+'training_data.txt',dtype=str)
        for pair in training_pairs:
          training_data.append([self.user_name_dict[pair[0]],self.item_name_dict[pair[1]],int(pair[2])])
        print('# training samples :', len(training_data))
        self.training_data = np.array(training_data)
        return True
    
    def sample_test(self):
        print('======================= sample test data =======================')
        user_item_label_list = []  # [[u, [item1, item2, ...], [l1, l2, ...]], ...]
        with open(save_path+'test_data.pickle', 'rb') as f:
            test_pairs= pickle.load(f)
        for user_id in test_pairs.keys():
          user=self.user_name_dict[user_id]
          items_ids=test_pairs[user_id][0]
          labels=test_pairs[user_id][1]
          items=np.array([self.item_name_dict[item] for item in items_ids])
          labels=np.array([float(label) for label in labels])
          user_item_label_list.append([user,items,labels])
        print('# test samples :', len(user_item_label_list))
        self.test_data = np.array(user_item_label_list)
        return True
    # def sample_training(self):
    #     print('======================= sample training data =======================')
    #     # print(self.user_feature_matrix.shape, self.item_feature_matrix.shape)
    #     training_data = []
    #     item_set = set(self.items)
    #     for user, items in self.user_hist_inter_dict.items():
    #         if len(items)>9:
    #           items = items[:-(test_length)]
    #           training_pairs = sample_training_pairs(
    #               user, 
    #               items, 
    #               item_set, 
    #               sample_ratio)
    #           for pair in training_pairs:
    #               training_data.append(pair)
    #     print('# training samples :', len(training_data))
    #     self.training_data = np.array(training_data)
    #     return True
    
    # def sample_test(self):
    #     print('======================= sample test data =======================')
    #     user_item_label_list = []  # [[u, [item1, item2, ...], [l1, l2, ...]], ...]
    #     for user, items in self.user_hist_inter_dict.items():
    #       if len(items)>9:
    #         items = items[-(test_length):]
    #         user_item_label_list.append([user, items, np.ones(len(items))])  # add the test items
    #         neg_length=len(items)*10
    #         negative_items = [item for item in self.items if 
    #             item not in self.user_hist_inter_dict[user]]  # the not interacted items
    #         negative_items = np.random.choice(np.array(negative_items), neg_length, replace=False)
    #         user_item_label_list[-1][1] = np.concatenate((user_item_label_list[-1][1], negative_items), axis=0)
    #         user_item_label_list[-1][2] = np.concatenate((user_item_label_list[-1][2], np.zeros(neg_length)), axis=0)
    #     print('# test samples :', len(user_item_label_list))
    #     self.test_data = np.array(user_item_label_list)
    #     return True

    def save(self, save_path):
        return True
    
    def load(self):
        return False

In [ ]:
def amazon_preprocessing():
    rec_dataset = AmazonDataset()
    return rec_dataset

In [ ]:
def dataset_init():
	if dataset == "yelp":
		rec_dataset = yelp_preprocessing()
	elif dataset == "cell_phones" or "kindle_store" or "electronic" or "cds_and_vinyl":
		rec_dataset = amazon_preprocessing()
	return rec_dataset

In [ ]:
dataset='cds_and_vinyl'
gpu=True
cuda='0'
weight_decay=0.00001
lr=0.01
epochs=100
batch_size=64
rec_k=5

In [ ]:
import numpy as np
from torch.utils.data import Dataset
class UserItemInterDataset(Dataset):
    def __init__(self, data, user_feature_matrix, item_feature_matrix):
        self.data = data
        self.user_feature_matrix = user_feature_matrix
        self.item_feature_matrix = item_feature_matrix

    def __getitem__(self, index):
        user = self.data[index][0]
        item = self.data[index][1]
        label = self.data[index][2]
        user_feature = self.user_feature_matrix[user]
        item_feature = self.item_feature_matrix[item]
        return user_feature, item_feature, label
    def __len__(self):
        return len(self.data)

# Train Black-box model...

In [ ]:
import numpy as np
import torch
from sklearn.metrics import ndcg_score

In [ ]:
def compute_ndcg(test_data, user_feature_matrix, item_feature_matrix, k, model, device):
    model.eval()
    ndcgs = []
    with torch.no_grad():
        for row in test_data:
            user = row[0]
            items = row[1]
            gt_labels = row[2]
            user_features = np.array([user_feature_matrix[user] for i in range(len(items))])
            item_features = np.array([item_feature_matrix[item] for item in items])
            scores = model(torch.from_numpy(user_features).to(device),
                                    torch.from_numpy(item_features).to(device)).squeeze()
            scores = np.array(scores.to('cpu'))
            ndcg = ndcg_score([gt_labels], [scores], k=k)
            ndcgs.append(ndcg)
    ave_ndcg = np.mean(ndcgs)
    return ave_ndcg


In [ ]:
import torch
import numpy as np
import os
import tqdm
import pickle
from pathlib import Path
from torch.utils.data import DataLoader

In [ ]:
from numpy import core

class BaseRecModel(torch.nn.Module):
    def __init__(self, feature_length):
        super(BaseRecModel, self).__init__()
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(feature_length * 2, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, user_feature, item_feature):
        fusion = torch.cat((user_feature, item_feature), 1)
        out = self.fc(fusion)
        return out


In [ ]:
if gpu:
  device = torch.device('cuda:%s' % cuda)
else:
  device = 'cpu'
print(device)

cuda:0


In [ ]:
rec_dataset = dataset_init()

1945
----- get items matrix -------
1945
0
200
400
600
800
1000
1200
1400
1600
1800
----- get users matrix -------
6794
0
1000
2000
3000
4000
5000
6000
======================= sample training data =======================
# training samples : 2871
======================= sample test data =======================
# test samples : 93


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
rec_dataset.item_feature_matrix.shape

(1945, 2304)

In [ ]:
with open(os.path.join(save_path, dataset + "_dataset_obj.pickle"), 'rb') as inp:
  rec_dataset = pickle.load(inp)

In [ ]:
def train_base_recommendation():
    if gpu:
        device = torch.device('cuda:%s' % cuda)
    else:
        device = 'cpu'
    Path(save_path).mkdir(parents=True, exist_ok=True)
    with open(os.path.join(save_path,dataset + "_dataset_obj.pickle"), 'wb') as outp:
        pickle.dump(rec_dataset, outp, pickle.HIGHEST_PROTOCOL)

    train_loader = DataLoader(dataset=UserItemInterDataset(rec_dataset.training_data, 
                                rec_dataset.user_feature_matrix, 
                                rec_dataset.item_feature_matrix),
                          batch_size=batch_size,
                          shuffle=True)

    model = BaseRecModel(rec_dataset.feature_num).to(device)
    loss_fn = torch.nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)

    out_path = os.path.join("./logs", dataset + "_logs")
    Path(out_path).mkdir(parents=True, exist_ok=True)

    ndcg = compute_ndcg(rec_dataset.test_data, 
            rec_dataset.user_feature_matrix, 
            rec_dataset.item_feature_matrix, 
            rec_k, 
            model, 
            device)
    print('init ndcg:', ndcg)
    for epoch in tqdm.trange(epochs):
        model.train()
        optimizer.zero_grad()
        losses = []
        for user_behaviour_feature, item_aspect_feature, label in train_loader:
            user_behaviour_feature = user_behaviour_feature.to(device)
            item_aspect_feature = item_aspect_feature.to(device)
            label = label.float().to(device)
            out = model(user_behaviour_feature, item_aspect_feature).squeeze()
            loss = loss_fn(out, label)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            losses.append(loss.to('cpu').detach().numpy())
            ave_train = np.mean(np.array(losses))
        print('epoch %d: ' % epoch, 'training loss: ', ave_train)
        # compute necg
        if epoch % 10 == 0:
            ndcg = compute_ndcg(rec_dataset.test_data, 
            rec_dataset.user_feature_matrix, 
            rec_dataset.item_feature_matrix, 
            rec_k, 
            model, 
            device)
            print('epoch %d: ' % epoch, 'training loss: ', ave_train, 'NDCG: ', ndcg)
    torch.save(model.state_dict(), os.path.join(save_path, "model.model"))
    return rec_dataset


if __name__ == "__main__":
    torch.manual_seed(0)
    np.random.seed(0)
    if gpu:
        os.environ["CUDA_VISIBLE_DEVICES"] =cuda
        print("Using CUDA",cuda)
    else:
        print("Using CPU")
    rec_dataset=train_base_recommendation()

Using CUDA 0
init ndcg: 0.07731034104130662


  0%|          | 0/240 [00:00<?, ?it/s]

epoch 0:  training loss:  0.6559843


  0%|          | 1/240 [00:00<02:20,  1.71it/s]

epoch 0:  training loss:  0.6559843 NDCG:  0.08904614145749981


...

epoch 235:  training loss:  0.18343328
epoch 236:  training loss:  0.16697739


100%|█████████▉| 239/240 [00:57<00:00,  6.05it/s]

epoch 237:  training loss:  0.16849056
epoch 238:  training loss:  0.21604227


100%|██████████| 240/240 [00:57<00:00,  4.16it/s]


epoch 239:  training loss:  0.24283147


In [ ]:
dataset='cds_and_vinyl'
base_model_path="/content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/cds/"
gpu=True
cuda='0'
data_obj_path="/content/drive/Shareddrives/Unlimited Drive | @LicenseMarket/Recommender/cds/"
rec_k=5
lam=100
gam=0.7
alp=0.2
user_mask=False
lr=0.01
step=500
mask_thresh=0.1
test_num=-1
# save_path="./explanation_objs/"

In [ ]:
import torch
import pickle
import os
from pathlib import Path

# User Perspective Features

In [ ]:
import json
import nltk

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def lemmatization(text):
    result=[]
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
        # if pos in ['n','a']:   
        #   result.append(wordnet.lemmatize(token,pos))
    return result
def remove_stopwords(text):
    en_stopwords = stopwords.words('english')
    en_stopwords+=['may','could','that','without','iii','with','and','This','That','Those','These','the','The','brbr','so','it','such']
    result = []
    for token in text:
        if token not in en_stopwords:
            result.append(token)
            
    return result
def remove_punct(text):
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

def remove_tag(text):
    text=' '.join(text)
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def preprocess(text):
  chars=['&','%','#','@','^','>','<','\n','\\','\t',';','"','/']
  stwords=stopwords.words('english')
  for ch in chars:
    text=text.replace(ch,' ')
  text=" ".join(text.split())
  # text=text.lower()
  text_tokenized=word_tokenize(text)
  cleaned_text= remove_stopwords(text_tokenized)
  cleaned_text= remove_punct(cleaned_text)
  # cleaned_text=lemmatization(cleaned_text)
  cleaned_text=remove_tag(cleaned_text)
  cleaned_text=remove_urls(cleaned_text)
  cleaned_text=''.join([i for i in cleaned_text ])
  cleaned_text=[word for word in cleaned_text.split(' ') if len(word)>1]
  # print(cleaned_text)
  return ' '.join(cleaned_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def preprocess_text_first(text):
  list_to_replace=['br','mso','gte','xml','false','!','-','\'','\"','[',']','/','\\n','\\','span','a-size-base','a-color-secondary','input type','header name','value','=','<a href= javascript:void(0) class= ','{','}','class=','header','<a href= javascript:void(0)','<','>','href',')','(',';','quot','&',':','javascript']
  for char in list_to_replace:
    text=text.replace(char,' ')
  for i in range(15):
    text=text.replace('  ',' ')
  new_text=''
  for word in text.split(' '):
    if len(word)>1 and len(word)<35:
      new_text+=word+' '
  return new_text

In [ ]:
items_list=[]
users_list=[]
review_features={}
f=open(save_path+'CDs_and_Vinyl')
lines=f.readlines()
i=0
for line in lines:
  if i%5000==0:
    print(i)
  i+=1
  user_id = line.split('@')[0]
  item_id = line.split('@')[1]
  # if item_id in df_meta['asin'].values:
  users_list.append(user_id)
  items_list.append(item_id)
  l = len(user_id) + len(item_id)
  fosr_data = line[l+3:]
  for seg in fosr_data.split('||'):
    if (user_id,item_id) not in review_features.keys():
      review_features[(user_id,item_id)]=[]
    fos = seg.split(':')[0].strip('|')
    if len(fos.split('|')) > 1:
          feature = fos.split('|')[0]
          opinion = fos.split('|')[1]
          sentiment = fos.split('|')[2]
          sentence= seg.split(':')[1].lower()
          if sentiment=='+1':
            senti=1
          else:
            senti=-1
          review_features[(user_id,item_id)].append([feature,opinion,senti,sentence])

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000


In [ ]:
user_test_perspective={}
i=0
for (user_id , item_id) in review_features.keys():
  review_feature=review_features[(user_id,item_id)]
  if i%5000==0:
    print(i)
  i+=1
  for features in review_feature:
    sentence=features[3]
    sentence=preprocess_text_first(sentence)
    sentence=preprocess(sentence).lower()
    if (user_id , item_id) not in user_test_perspective.keys():
      final_vect=[]
    final_vect+=sentence.split(' ')
    for word in sentence.split(' '):
      # tokens=list(set(df_words[df_words['word']==word]['tokenized'].values))
      # for token in tokens:
        final_vect+=word.split(' ')
    final_vect=list(set(final_vect))
    new_final_vect=[]
    for word in final_vect:
      if len(word)>1:
        new_final_vect.append(word)
    user_test_perspective[(user_id , item_id)]=new_final_vect

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000


In [ ]:
user_test_perspective

In [ ]:
rec_dataset.inv_item_name_dict[622]

'B00FL6ADNC'

# Train Explaination Generator Model


In [ ]:
def get_features(item_id):
  item=rec_dataset.item_name_dict[item_id]
  item_features=rec_dataset.item_features[item]
  len_features=0
  features=[]
  th1=len(item_features[0])
  # th2=len(item_features[0])+len(item_features[1])
  for i in range(2):
    len_features+=len(item_features[i])
    features+=item_features[i]
  return features,th1

In [ ]:
def get_average_vect_test(weights,tensor_vect):
  weights=torch.FloatTensor(weights).to(device)
  # print()
  weights=torch.transpose(weights.repeat(tensor_vect.shape[1],1),0,1)
  average=torch.mean(weights*tensor_vect,axis=0).to(device)
  return average

In [ ]:
def get_new_item_vector(item_id,th1,new_weights,tensor_vect_desc,tensor_vect_title):
    # item_id= rec_dataset.inv_item_name_dict[item]
    final_vector=[]
    average_vect_desc=get_average_vect_test(new_weights[0:th1],tensor_vect_desc)
    average_vect_title=get_average_vect_test(new_weights[th1:],tensor_vect_title)

    final_vector=torch.cat((average_vect_desc,average_vect_title))
      # final_vect=np.array(final_vector, dtype='float32')
    return final_vector

In [ ]:
def get_random_chromosome(features):
  # print(features)
  ch = np.random.choice([0, 1], size=len(features),p=[0.1,0.9])
  # for i in range(10):
  #   rand_num=np.random.randint(0,len(ch))
  #   ch[rand_num]=0
  # ch=np.ones(len(features))
  return ch

In [ ]:
def get_first_population(num_population,features):
  pops=[]
  for i in range(num_population):
    ch=get_random_chromosome(features)
    pops.append(ch)
  return pops

In [ ]:
def select_chrs(dict_chooses,num_population,population):
  new_population=[]
  while len(new_population)<num_population:
    rand_num=np.random.rand()
    for key in dict_chooses.keys():
      if rand_num>=key[0] and rand_num<=key[1]:
        new_population.append(population[dict_chooses[key]])
  return new_population

In [ ]:
alpha=1
lam=10
beta=10

In [ ]:
def calc_fitnesses(margin_score,population,th1,user_id,item_id,base_model,tensor_vect_desc,tensor_vect_title):
  best_fit=-10000.0
  best_ans=[]
  dict_fitnesses={}
  # print(out_last)
  count=0
  # print(len(population))
  user=rec_dataset.user_name_dict[user_id]
  user_vect=torch.from_numpy(rec_dataset.user_feature_matrix[user]).to(device)
  count_best=0
  for chr in population:
    item_feature_star=get_new_item_vector(item_id,th1,chr,tensor_vect_desc,tensor_vect_title)
    score = base_model(user_vect.unsqueeze(0), item_feature_star.unsqueeze(0))
    count_features_rate=np.count_nonzero(chr)/len(chr)
    beta=10
    if score.cpu()-margin_score.cpu()>0:
      count_features_rate=(len(chr)-np.count_nonzero(chr))/len(chr)
      beta=0.5
      
    # else:
    #   alpha=100.0
    #   beta=1.0
    # print(margin_score)
    # print(score)
    fitness=1.0/(lam*(((alpha+score.cpu()-margin_score.cpu()))))+(beta*((count_features_rate)))
    # if out_new<0.5:
    #   for rate in [0.8,0.7,0.6,0.5,0.4,0.3]:
    #     if (np.count_nonzero(chr)/len(chr))<rate:
    #       fitness-=(1.0-rate)
    #       break
    if fitness>best_fit:
      best_fit=fitness
      best_ans=chr.copy()
      out_best_last=margin_score
      out_best_new=score
      count_best=len(chr)-np.count_nonzero(chr)
    # print(fitness,np.log((out_last-out_new)+1.0),((np.count_nonzero(chr)/len(chr))))
    dict_fitnesses[count]=fitness.item()
    count+=1
  # print(len(dict_fitnesses))
  # print('score: ',out_best_new)
  # print(len(best_ans)-np.count_nonzero(best_ans),len(best_ans))
  # print('fitness: ',best_fit)
  # print('----------')
  return dict_fitnesses,best_fit,best_ans,out_best_new,count_best

In [ ]:
def cross_over(th1,new_pop1):
  new_pop2=[]
  # print(len(new_pop1))
  # print()
  for i in range(0,len(new_pop1)-1,2):
    chr1=new_pop1[i]
    chr2=new_pop1[i+1]
    rand_num=np.random.rand()
    if rand_num>0.01:
      new_chr1=list(chr1[0:th1])
      new_chr1+=list(chr2[th1:])
      # new_chr1+=list(chr1[th2:])

      new_chr2=list(chr2[0:th1])
      new_chr2+=list(chr1[th1:])
      # new_chr2+=list(chr2[th2:])

      new_pop2.append(new_chr1)
      new_pop2.append(new_chr2)
    else:
      new_pop2.append(chr1)
      new_pop2.append(chr2)
  return new_pop2

In [ ]:
def mutate(new_pop):
  new_pop1=new_pop.copy()
  # print(len(new_pop1))
  # print(len(new_pop))
  rand_count=np.random.randint(0,0.5*int(len(new_pop1)))
  for i in range(rand_count):
    rand_ind=np.random.randint(0,len(new_pop1))
    chr1=new_pop1[rand_ind]
    rand_count2=np.random.randint(0,0.1*int(len(new_pop1)))
    for j in range(rand_count2):
      rand_ind=np.random.randint(0,len(chr1))
      chr1[rand_ind]=1.0-chr1[rand_ind]
  return new_pop1

In [ ]:
num_population=200

In [ ]:
def evaluate_model_perspective(
        rec_dict,
        u_i_exp_dict,
        base_model,
        user_feature_matrix,
        item_feature_matrix,
        rec_k,
        device):
    """
    compute PN, PS and F_NS score for the explanations
    :param rec_dict: {u1: [i1, i2, i3, ...] , u2: [i1, i2, i3, ...]}
    :param u_i_exp_dict: {(u, i): [f1, f2, ...], ...}
    :param base_model: the trained base recommendation model
    :param user_feature_matrix: |u| x |p| matrix, the attention on each feature p for each user u
    :param item_feature_matrix: |i| x |p| matrix, the quality on each feature p for each item i
    :param rec_k: the length of the recommendation list, only generated explanations for the items on the list
    :param device: the device of the model
    :return: the mean of the PN, PS and FNS scores
    """
    pn_count = 0
    ps_count = 0
    dict_items_cf_feature={}
    for u_i, fs in u_i_exp_dict.items():
        user = u_i[0]
        target_item = u_i[1]
        features = set(fs)
        items = rec_dict[user]
        target_index = items.index(target_item)
        # print(len(items))
        # print(features)
        # compute PN
        cf_items_features1 = []
        cf_items_features2 = []
        for item in items:
            # print(item)
            # item_ori_feature = np.array(item_feature_matrix[item])
            item_id=rec_dataset.inv_item_name_dict[item]
            all_features,th1=get_features(item_id)
            # print(all_features)
            item_feature_name=rec_dataset.item_features[item]
            weights1=[0.0 if fea in features else 1.0 for fea in all_features]
            weights2=[1.0 if fea in features else 0.0 for fea in all_features]
            # weights=torch.FloatTensor(weights).to(device)
            if item in dict_items_cf_feature.keys():
              tensor_vect_desc,tensor_vect_title=dict_items_cf_feature[item]
            else:
              tensor_vect_desc,tensor_vect_title=get_tensor_vects(item_id)
              dict_items_cf_feature[item]=(tensor_vect_desc,tensor_vect_title)
            
            item_cf_feature1=get_new_item_vector(item_id,th1,weights1,tensor_vect_desc,tensor_vect_title).detach().to('cpu').numpy()
            item_cf_feature2=get_new_item_vector(item_id,th1,weights2,tensor_vect_desc,tensor_vect_title).detach().to('cpu').numpy()
            cf_items_features1.append(item_cf_feature1)
            cf_items_features2.append(item_cf_feature2)
            # print(item_cf_feature)
            # print(torch.from_numpy(np.array(cf_items_features,dtype='float32')).to(device))
            # print(np.shape(item_cf_feature))
            # print(np.shape(cf_items_features))
        # print(np.shape(cf_items_features))
        cf_ranking_scores1 = base_model(torch.from_numpy(np.array([user_feature_matrix[user]
                                                                      for i in range(len(cf_items_features1))])
                                                            ).to(device),
                                           torch.from_numpy(np.array(cf_items_features1,dtype='float32')).to(device)).squeeze()
        cf_score_list1 = cf_ranking_scores1.to('cpu').detach().numpy()
        sorted_index1 = np.argsort(cf_score_list1)[::-1]
        cf_rank1 = np.argwhere(sorted_index1 == target_index)[0, 0]  # the updated ranking of the current item
        if cf_rank1 > rec_k - 1:
            pn_count += 1
        # compute NS
        cf_ranking_scores2 = base_model(torch.from_numpy(np.array([user_feature_matrix[user]
                                                                      for i in range(len(cf_items_features2))])
                                                            ).to(device),
                                           torch.from_numpy(np.array(cf_items_features2,dtype='float32')).to(device)).squeeze()
        cf_score_list2 = cf_ranking_scores2.to('cpu').detach().numpy()
        sorted_index2 = np.argsort(cf_score_list2)[::-1]
        cf_rank2 = np.argwhere(sorted_index2 == target_index)[0, 0]  # the updated ranking of the current item
        if cf_rank2 < rec_k:
            ps_count += 1
    if len(u_i_exp_dict) != 0:
        pn = pn_count / len(u_i_exp_dict)
        ps = ps_count / len(u_i_exp_dict)
        if (pn + ps) != 0:
            fns = (2 * pn * ps) / (pn + ps)
        else:
            fns = 0
    else:
        pn = 0
        ps = 0
        fns = 0
    return pn, ps, fns

In [ ]:
def evaluate_user_perspective(user_perspective_data, u_i_expl_dict):
    pres = []
    recs = []
    f1s = []
    for u_i, gt_features in user_perspective_data.items():
        if u_i in u_i_expl_dict:
            TP = 0
            pre_features = u_i_expl_dict[u_i]
            # print('f: ', gt_features, pre_features)
            for feature in pre_features:
                if feature in gt_features:
                    TP += 1
            # print(gt_features)
            # print(pre_features)
            pre = TP / len(pre_features)
            rec = TP / len(gt_features)
            if (pre + rec) != 0:
                f1 = (2 * pre * rec) / (pre + rec)
            else:
                f1 = 0
            pres.append(pre)
            recs.append(rec)
            f1s.append(f1)
    ave_pre = np.mean(pres)
    ave_rec = np.mean(recs)
    ave_f1 = np.mean(f1s)
    return ave_pre, ave_rec, ave_f1

In [ ]:
class ExpOptimizationModel(torch.nn.Module):
    def __init__(self, base_model, rec_dataset, device):
        super(ExpOptimizationModel, self).__init__()
        self.base_model = base_model
        self.rec_dataset = rec_dataset
        self.device = device
        self.u_i_exp_dict = {}  # {(user, item): [f1, f2, f3 ...], ...}
        self.user_feature_matrix = torch.from_numpy(self.rec_dataset.user_feature_matrix).to(self.device)
        self.item_feature_matrix = torch.from_numpy(self.rec_dataset.item_feature_matrix).to(self.device)
        self.rec_dict, self.user_perspective_test_data = self.generate_rec_dict()

    def generate_rec_dict(self):
        rec_dict = {}
        correct_rec_dict = {}  # used for user-side evaluation
        for row in self.rec_dataset.test_data:
            user = row[0]
            items = row[1]
            labels = row[2]
            correct_rec_dict[user] = []
            user_features = self.user_feature_matrix[user].repeat(len(items), 1)
            scores = self.base_model(user_features,
                        self.item_feature_matrix[items]).squeeze()
            scores = np.array(scores.to('cpu'))
            sort_index = sorted(range(len(scores)), key=lambda k: scores[k], reverse=True)
            sorted_items = [items[i] for i in sort_index]
            rec_dict[user] = sorted_items
            for i in range(rec_k):  # find the correct items and add to the user side test data
                if labels[sort_index[i]] == 1:
                    correct_rec_dict[user].append(items[sort_index[i]])

        user_perspective_test_data = {}  # {(u, i):f, (u, i): f]}
        # for user, items in correct_rec_dict.items():
        #     for item in items:
        #         user_id=rec_dataset.inv_user_name_dict[user]
        #         item_id=rec_dataset.inv_item_name_dict[item]
        #         feature = user_test_perspective[(user_id, item_id)]
        #         user_perspective_test_data[(user, item)] = feature
        return rec_dict, user_perspective_test_data
    def user_side_evaluation(self):
        ave_pre, ave_rec, ave_f1 = evaluate_user_perspective(self.user_perspective_test_data, self.u_i_exp_dict)
        print('user\'s perspective:')
        print('ave pre: ', ave_pre, '  ave rec: ', ave_rec, '  ave f1: ', ave_f1)
    
    def model_side_evaluation(self):
        ave_pn, ave_ps, ave_fns = evaluate_model_perspective(
            self.rec_dict,
            self.u_i_exp_dict,
            self.base_model,
            self.rec_dataset.user_feature_matrix,
            self.rec_dataset.item_feature_matrix,
            rec_k,
            self.device)
        print('model\'s perspective:')
        print('ave PN: ', ave_pn, '  ave PS: ', ave_ps, '  ave F_{NS}: ', ave_fns)  
    def generate_explanation(self):
        # u_i_exps_dict = {}  # {(user, item): [f1, f2, f3 ...], ...}
        exp_nums = []
        exp_complexities = []
        self.no_exp_count = 0
        # test_num=10
        if test_num == -1:
            test_num1 = len(list(self.rec_dict.items()))
        else:
            test_num1 = test_num
        count=0
        for user, items in tqdm.tqdm(list(self.rec_dict.items())[:20]):
            user_id=self.rec_dataset.inv_user_name_dict[user]
            count+=1
            # if count<200:
            #   continue
            # if count==3:
            #   break
            items = self.rec_dict[user]
            margin_item = items[rec_k]
            margin_score = self.base_model(self.user_feature_matrix[user].unsqueeze(0), 
                            self.item_feature_matrix[margin_item].unsqueeze(0)).squeeze()
            # print('margin_score: ',margin_score)
            for item in items[: rec_k]:
                item_id=self.rec_dataset.inv_item_name_dict[item]
                tensor_vect_desc,tensor_vect_title=get_tensor_vects(item_id)
                explanation_features_words, exp_num=self.get_explanation(item_id,margin_score,user_id,tensor_vect_desc,tensor_vect_title)
                
                if explanation_features_words is None:
                    # print('no explanation for user %d and item %d' % (user, item))
                    self.no_exp_count += 1
                else:
                    self.u_i_exp_dict[(user, item)] = explanation_features_words
                    # print(explanation_features_words)
                    exp_nums.append(exp_num)

              
        print('ave num: ', np.mean(exp_nums), 'ave complexity: ', np.mean(exp_complexities) , 'no_exp_count: ', self.no_exp_count)
        return True
    def get_explanation(self,item_id,margin_score,user_id,tensor_vect_desc,tensor_vect_title):
      best_answer=[]
      best_fit_all=-10000.0
      features,th1=get_features(item_id)
      # print(features)
      population=get_first_population(num_population,features)
      # print(len(population))
      # print(population)
      exit=False
      iter=0
      while not exit:
        fitnesses,best_fit,best_ans,best_score,count_zero= calc_fitnesses(margin_score,population,th1,user_id,item_id,self.base_model,tensor_vect_desc,tensor_vect_title)
        # print(population[0])
        # print(fitnesses)
        # if iter%1==0:
        #   print('iter: ', iter, 'best_fit: ',best_fit , 'best_score: ', best_score , 'count_fea: ', count_zero )
        iter+=1
        if best_fit>best_fit_all:
          best_fit_all=best_fit
          best_answer=best_ans.copy()
        dict_probs={}
        for key in fitnesses.keys():
          # print(fitnesses[key])
          # print(np.sum(list(fitnesses.values())))
          dict_probs[key]=fitnesses[key]/np.sum(list(fitnesses.values()))
        dict_chooses={}
        last_prob=0
        sum_prob=0
        for key in dict_probs.keys():
          prob=dict_probs[key]
          dict_chooses[(sum_prob,prob+sum_prob)]=key
          sum_prob+=prob
        # print(dict_chooses)
        new_pop1=select_chrs(dict_chooses,num_population,population)
        # print(len(new_pop1))
        new_pop2=cross_over(th1,new_pop1)
        # print(len(new_pop2))
        rand_num=np.random.rand()
        if rand_num>0.9:
          new_pop3=mutate(new_pop2)
          # print(len(new_pop3))
        else:
          new_pop3=new_pop2.copy()
        population=new_pop3.copy()
        if best_fit_all.cpu()>=1.0 and iter==10:
          exit=True 
        elif iter==50:
          exit=True
      if best_fit_all.cpu()>=1.0:
        best_features=[features[i] for i in range(len(features)) if best_answer[i]<1]
        # print(best_features)
        return best_features,len(best_features)
      else:
        # print('no explanation')
        return None,None

In [ ]:
def get_tensor_vect_df(df,not_in_columns):
  df1=df.copy()
  df1['main_word']=np.where(df1['replaced_word'] == df1[not_in_columns],1,0)
  df1=df1[df1['main_word']==1]
  if df1.empty:
    df1=df.copy()
    df1['main_word']=np.where(df1['logit']==5.0,1,0)
    df1=df1[df1['main_word']==1]
  if df1.empty:
    df1=df.copy()
    df1['main_word']=np.where(df1['logit']==12.0,1,0)
    df1=df1[df1['main_word']==1]
  lists=df1.loc[:, ~df1.columns.isin([not_in_columns,'replaced_word','logit','main_word'])].values
  words=df1[not_in_columns].values
  # print(words)
  vects=[sub_list[0] for sub_list in lists]
  tensor_vect=torch.FloatTensor(vects).to(device)
  return tensor_vect

In [ ]:
def get_tensor_vects(item_id):
  tensor_vect_desc=None
  tensor_vect_title=None
  # print(item_id)
  # if os.path.exists(save_path+'descriptions_bert/'+'df_bert_desc_{}.json'.format(item_id)) :
  df_vect_desc= pd.read_json(save_path+'descriptions_bert/'+'{}.json'.format(item_id))
  # df_vect_desc=df_vect[0]
  tensor_vect_desc=get_tensor_vect_df(df_vect_desc,'description_words')

  df_vect_title= pd.read_json(save_path+'titles_bert/'+'{}.json'.format(item_id))
  # df_vect_title=df_vect[1]
  tensor_vect_title=get_tensor_vect_df(df_vect_title,'title_words')
  return tensor_vect_desc,tensor_vect_title

In [ ]:
with open(os.path.join(data_obj_path, dataset + "_dataset_obj.pickle"), 'rb') as inp:
        rec_dataset = pickle.load(inp)

In [ ]:
def generate_explanation():
    if gpu:
        device = torch.device('cuda:%s' %cuda)
    else:
        device = 'cpu'
    print(device)
    # import dataset
    with open(os.path.join(data_obj_path, dataset + "_dataset_obj.pickle"), 'rb') as inp:
        rec_dataset = pickle.load(inp)
    base_model = BaseRecModel(rec_dataset.feature_num).to(device)
    base_model.load_state_dict(torch.load(os.path.join(base_model_path,"model.model"),map_location=torch.device(device)))
    base_model.eval()
    #  fix the rec model
    for param in base_model.parameters():
        param.requires_grad = False
    
    # Create optimization model
    opt_model = ExpOptimizationModel(
        base_model=base_model,
        rec_dataset=rec_dataset,
        device = device,
        
    )

    opt_model.generate_explanation()
    Path(save_path).mkdir(parents=True, exist_ok=True)
    with open(os.path.join(save_path, dataset + "_explanation_obj_genetic.pickle"), 'wb') as outp:
        pickle.dump(opt_model, outp, pickle.HIGHEST_PROTOCOL)
    # with open(os.path.join(save_path, dataset + "_explanation_obj.pickle"), 'rb') as opt:
    #     opt_model = pickle.load(opt)
    opt_model.user_side_evaluation()
    opt_model.model_side_evaluation()
    # print(opt_model.u_i_exp_dict)
    # Path(save_path).mkdir(parents=True, exist_ok=True)
    # with open(os.path.join(save_path, dataset + "_explanation_obj.pickle"), 'wb') as outp:
    #     pickle.dump(opt_model, outp, pickle.HIGHEST_PROTOCOL)
    return opt_model


if __name__ == "__main__":
    opt_model=generate_explanation()

cuda:0


100%|██████████| 6931/6931 [17:57:51<00:00,  9.33s/it]
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ave num:  7.328972238629651 ave complexity:  nan no_exp_count:  795
user's perspective:
ave pre:  0.12181757349902651   ave rec:  0.034752825736500854   ave f1:  0.03847731861005478
model's perspective:
ave PN:  0.9578854105138807   ave PS:  0.9971647962197283   ave F_{NS}:  0.9771305175561397


# calculate Stability...

In [ ]:
def generate_explanation_check_stability():
    if gpu:
        device = torch.device('cuda:%s' %cuda)
    else:
        device = 'cpu'
    print(device)
    # import dataset
    with open(os.path.join(data_obj_path, dataset + "_dataset_obj_2.pickle"), 'rb') as inp:
        rec_dataset = pickle.load(inp)
    
    base_model = BaseRecModel(rec_dataset.feature_num).to(device)
    base_model.load_state_dict(torch.load(os.path.join(base_model_path,"model2.model"),map_location=torch.device(device)))
    base_model.eval()
    #  fix the rec model
    for param in base_model.parameters():
        param.requires_grad = False
    
    # Create optimization model
    features_found=[]
    for i in range(10):
      opt_model = ExpOptimizationModel(
        base_model=base_model,
        rec_dataset=rec_dataset,
        device = device,)
      opt_model.generate_explanation()
      features_found.append(opt_model.u_i_exp_dict)
      
    
    return features_found


if __name__ == "__main__":
    features_found=generate_explanation_check_stability()

cuda:0


100%|██████████| 20/20 [05:47<00:00, 17.38s/it]
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ave num:  7.123711340206185 ave complexity:  nan no_exp_count:  3


100%|██████████| 20/20 [02:56<00:00,  8.84s/it]


ave num:  6.428571428571429 ave complexity:  nan no_exp_count:  2


...

ave num:  6.26530612244898 ave complexity:  nan no_exp_count:  2


100%|██████████| 20/20 [02:56<00:00,  8.83s/it]


ave num:  5.90625 ave complexity:  nan no_exp_count:  4


100%|██████████| 20/20 [02:37<00:00,  7.85s/it]

ave num:  7.393939393939394 ave complexity:  nan no_exp_count:  1


In [ ]:
with open(os.path.join(data_obj_path, dataset + "_dataset_obj_2.pickle"), 'rb') as inp:
     rec_dataset = pickle.load(inp)

In [ ]:
dict_features={}
for iter_feas in features_found:
  for u_i in iter_feas.keys():
    if u_i in dict_features.keys():
      dict_features[u_i].append(iter_feas[u_i])
    else:
      dict_features[u_i]=[]
      dict_features[u_i].append(iter_feas[u_i])

In [ ]:
dict_features

{(0, 11246): [['holiday', 'wishes'],
  ['holiday', 'wishes'],
  ['holiday', 'wishes'],
  ['holiday', 'wishes'],
  ['holiday', 'wishes'],
  ['holiday', 'wishes'],
  ['holiday', 'wishes'],
  ['wishes', 'holiday', 'wishes'],
  ['holiday', 'wishes'],
  ['holiday', 'wishes']],
 (0,
  9121): [['cappella',
   'christmas',
   'lsquo',
   'appropriately',
   'holiday',
   'holiday',
   'alongside',
   'holiday',
   'coming',
   'chaser',
   'snow',
   'christmas',
   'cheers'], ['album',
   'new',
   'world',
   'season',
   'studio',
   'sticking',
   'classic',
   'list',
   'donde',
   'gentlemen',
   'bells',
   'christmas',
   'cheers'], ['guys',
   'cities',
   'holiday',
   'today',
   'christmas',
   'cheers',
   'comedic',
   'time',
   'another',
   'christmastime',
   'snow',
   'rest',
   'christmas',
   'cheers'], ['2007',
   'together',
   'christmas',
   'lsquo',
   '110',
   'formula',
   'cheer',
   'track',
   'claus',
   'nosed',
   'mean',
   'christmas',
   'cheers'], ['200

In [ ]:
stability=0
# count_all=0
for ui in dict_features.keys():
  features=dict_features[ui]
  stabs=0
  count=0
  if(len(features)>1):
    # count_all+=1
    for i in range(len(features)):
      for j in range(len(features)):
        if i != j:
          intersection = list(set(features[i]) & set(features[j]))
          union = list(set(features[i]) | set(features[j]))
          # print(features[i],features[j])
          # print(intersection)
          # print(union)
          count+=1
          stabs+=(len(intersection)/len(union))
    # print(stabs)
    # print(len(features)*(len(features)-1))
    # print((stabs/(len(features)*(len(features)-1))))
    stability+=(stabs/(9.0*10.0))

stability=stability/len( dict_features)
print(stability)

0.4726154221190375
